# Resources


- [https://groups.google.com/forum/#!searchin/mtadeveloperresources/turnstile\$20december\$202014/mtadeveloperresources/9iuZg8ek9mE/amsfZ_OdHk0J](https://groups.google.com/forum/#!searchin/mtadeveloperresources/turnstile$20december$202014/mtadeveloperresources/9iuZg8ek9mE/amsfZ_OdHk0J)

# Imports


In [ ]:
# To handle I/O:
import os

# To handle dates:
from datetime import date,time,datetime,timedelta

# To handle data:
import numpy as np
import pandas as pd

# To create progress bars:
from tqdm import tqdm_notebook as tqdm


# Helper functions


In [ ]:
def delta_ddhhmmss(delta):
    t = delta.total_seconds()
    sign = "-" if t<0 else "+"
    t = -t if t<0 else t
    ss = int(t%60)
    mm = int(t%(60*60)//(60))
    hh = int(t%(24*60*60)//(60*60))
    dd = int(t//(24*60*60))
    dd = "{}d ".format(dd) if dd!=0 else ""
    return "{} {}{:0>2}h {:0>2}m {:0>2}s".format(sign,dd,hh,mm,ss).replace(' ','')


In [ ]:
def remove_outliers(data,grp_cols,data_cols):
    """
        Identifies values that are more than 2 standard deviations from the mean 
        (in their respective groups) and replaces them with np.nan .
    """
    
    #df = data[grp_cols+data_cols].copy()
    df = data.copy()
    
    ## Make all values positive:
    #for col in data_cols:
    #    df[col] = np.abs(df[col])
    
    progress_bar = tqdm(total=len(df.drop_duplicates(grp_cols)),desc="Remove outliers")
    
    new_df = []
    
    for i,grp in df.groupby(grp_cols,as_index=False):
        
        grp = grp.copy()
        
        for col in data_cols:
            
            mu = grp[col].mean()
            sd = grp[col].std()

            grp.loc[ (grp[col]<(mu-2*sd)) , col] = np.nan
            grp.loc[ (grp[col]>(mu+2*sd)) , col] = np.nan
            
        new_df.append(grp)
        
        progress_bar.update()
        
    progress_bar.close()
    
    new_df = pd.concat(new_df,axis=0)
    
    return new_df


# Lookups


In [ ]:
lookup_groupings = pd.read_csv('lookups/lookup_groupings.csv')
lookup_stations = pd.read_csv('lookups/lookup_stations.csv')
lookup_booths = pd.read_csv('lookups/lookup_booths.csv')


In [ ]:
lookup_booths

# Raw data


In [ ]:
source = []
for filename in os.listdir('data/'):
    if (filename.find('mta-turnstile-data_')>-1) and (filename.find('.csv')>-1):
        date_string = filename.replace('mta-turnstile-data_','').replace('.csv','')
        dt = datetime.strptime(date_string,'%Y-%m-%d').date()
        style = 'pre-2014-10-18' if dt<datetime(2014,10,18).date() else 'starting-2014-10-18'
        filepath = 'data/'+filename
        source.append({'dt':dt,'style':style,'filepath':filepath})
source = pd.DataFrame(source,columns=['dt','style','filepath']).sort_values(['dt'])

filepaths = list(source[source['style']=='starting-2014-10-18']['filepath'])
source[source['style']=='starting-2014-10-18']


# Intermediate data


In [ ]:
#filepaths = source[(source['dt']>=datetime(2017,1,1).date())&(source['dt']<datetime(2018,1,1).date())]
filepaths = source[source['dt']>=datetime(2018,1,1).date()]['filepath']
progress_bar = tqdm(total=len(filepaths))
for filepath in filepaths:
    raw = pd.read_csv(filepath)
    print('Read : {} .'.format(filepath))
    raw.columns = [c.strip() for c in raw.columns]
    for i,grp in raw.groupby(['C/A','UNIT']):
        ca = i[0]
        unit = i[1]
        filepath = 'data/intermediate/CA-BOOTH_{}-{}.csv'.format(ca,unit)
        is_first = not os.path.exists(filepath)
        with open(filepath,'a') as f:
            grp.to_csv(f,header=is_first,index=False)
    progress_bar.update()
progress_bar.close()


# Clean data


In [ ]:
data = []
filenames = [filename for filename in os.listdir('data/intermediate/') if filename.find('CA-BOOTH_')>-1]
progress_bar = tqdm(total=len(filenames))
for filename in filenames:
    filepath = 'data/intermediate/'+filename
    df = pd.read_csv(filepath)
    df = df.drop_duplicates()
    df['timestamp'] = [datetime.strptime("{} {}".format(d,t),"%m/%d/%Y %H:%M:%S") for d,t in zip(df['DATE'],df['TIME'])]
    
    for i,grp in df.groupby(['C/A','UNIT','SCP','STATION','LINENAME','DIVISION']):
        grp = grp.sort_values('timestamp')
        df1 = grp.iloc[:-1].reset_index(drop=True).copy()
        df2 = grp.iloc[1:].reset_index(drop=True).copy()
        # Get left (headers and start) columns:
        df1 = df1.rename(columns={
            'timestamp':'start_timestamp',
            'ENTRIES':'start_entries',
            'EXITS':'start_exits',
        }).drop(['DATE','TIME','DESC'],axis=1)
        # Get right (end) columns:
        df2 = df2.rename(columns={
            'timestamp':'end_timestamp',
            'ENTRIES':'end_entries',
            'EXITS':'end_exits',
        }).drop(['DATE','TIME','DESC','C/A','UNIT','SCP','STATION','LINENAME','DIVISION'],axis=1)
        # Merge left and right columns:
        df3 = pd.concat([df1,df2],axis=1)
        # Calculate deltas:
        df3['delta_entries'] = df3['end_entries'] - df3['start_entries']
        df3['delta_exits'] = df3['end_exits'] - df3['start_exits']
        df3['delta_timestamp'] = df2['end_timestamp']-df3['start_timestamp']
        df3['delta_timestamp'] = [delta_ddhhmmss(delta) for delta in df3['delta_timestamp']]
        # Append result:
        data.append(df3)
    
    progress_bar.update()

progress_bar.close()

data = pd.concat(data,axis=0)

data.head()


# Calculated data


In [ ]:
df1 = lookup_booths.copy()
df1 = df1[~pd.isnull(df1['Booth'])]
df1 = df1[['Complex ID','Booth']].drop_duplicates()
df1['Complex ID'] = [-1 if pd.isnull(x) else int(x) for x in df1['Complex ID']]
df1.head()


In [ ]:
df2 = data.copy()
df2['delta_entries'] = df2['delta_entries'].abs()
df2['delta_exits'] = df2['delta_exits'].abs()
df2 = remove_outliers(data=df2,grp_cols=['C/A','UNIT','SCP'],data_cols=['delta_entries','delta_exits'])
df2 = df2.groupby(['C/A','UNIT','start_timestamp','end_timestamp','delta_timestamp'])[['delta_entries','delta_exits']].sum()
df2.head()


In [ ]:
DATA = df1.merge(df2.reset_index(),left_on=['Booth'],right_on=['UNIT'],how='outer')
#DATA = DATA[DATA['delta_timestamp']=="+04h00m00s"]
DATA['Complex ID'] = [-1 if pd.isnull(x) else int(x) for x in DATA['Complex ID']]
DATA['Booth'] = ['?' if pd.isnull(x) else str(x) for x in DATA['Booth']]
DATA = DATA.groupby(['Complex ID','start_timestamp','end_timestamp','delta_timestamp'])[['delta_entries','delta_exits']].sum()
DATA['delta_entries'] = [int(x) for x in DATA['delta_entries']]
DATA['delta_exits'] = [int(x) for x in DATA['delta_exits']]
DATA = DATA.reset_index(drop=False)
DATA


In [ ]:
#data[['delta_entries','delta_exits']].sum()
#clean[['delta_entries','delta_exits']].sum()
#df2[['delta_entries','delta_exits']].sum()
#DATA[['delta_entries','delta_exits']].sum()


In [ ]:
DATA.to_csv('data/results/turnstile_data.csv',index=False)
print('Saved : data/results/turnstile_data.csv .')
